In [1]:
# https://hh.ru/search/vacancy?employment=probation&ored_clusters=true&hhtmFrom=vacancy_search_list&hhtmFromLabel=vacancy_search_line&search_field=name&search_field=company_name&search_field=description&enable_snippets=false&experience=noExperience&professional_role=156&professional_role=164&professional_role=165&professional_role=10&text=

In [3]:
!pip install selenium

  Obtaining dependency information for selenium from https://files.pythonhosted.org/packages/3f/fd/c2e7bb547b5b96c7bd536b4a80c4564b7ce5cd38d10095fbba8648996ab9/selenium-4.18.1-py3-none-any.whl.metadata
  Using cached selenium-4.18.1-py3-none-any.whl.metadata (6.9 kB)
  Obtaining dependency information for trio~=0.17 from https://files.pythonhosted.org/packages/14/fb/9299cf74953f473a15accfdbe2c15218e766bae8c796f2567c83bae03e98/trio-0.24.0-py3-none-any.whl.metadata
  Using cached trio-0.24.0-py3-none-any.whl.metadata (4.9 kB)
  Obtaining dependency information for trio-websocket~=0.9 from https://files.pythonhosted.org/packages/48/be/a9ae5f50cad5b6f85bd2574c2c923730098530096e170c1ce7452394d7aa/trio_websocket-0.11.1-py3-none-any.whl.metadata
  Using cached trio_websocket-0.11.1-py3-none-any.whl.metadata (4.7 kB)
  Obtaining dependency information for typing_extensions>=4.9.0 from https://files.pythonhosted.org/packages/b7/f4/6a90020cd2d93349b442bfcb657d0dc91eee65491600b2cb1d388bc98e6b/typ

In [4]:
from bs4 import BeautifulSoup
from pprint import pprint
from selenium import webdriver
import json
import pandas as pd

In [6]:
url = "https://hh.ru"
all_vacancies = []

In [7]:
driver = webdriver.Edge()

In [8]:
url2 = "https://hh.ru/search/vacancy?employment=probation&ored_clusters=true&hhtmFrom=vacancy_search_list&hhtmFromLabel=vacancy_search_line&search_field=name&search_field=company_name&search_field=description&enable_snippets=false&experience=noExperience&professional_role=156&professional_role=164&professional_role=165&professional_role=10&text="

In [9]:
while True:
    # response = session.get(url_param, params=params, headers=headers)
    driver.get(url2)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    vacancies = soup.find_all('div', {'class': 'vacancy-serp-item__layout'})

    for vacancy in vacancies:
        vac_info = {}

        vac_info['name'] = vacancy.find('span', {'class': 'serp-item__title'}).getText()

        vac_info['link'] = vacancy.find('a').get('href')

        try:
            salary_info = vacancy.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'}).getText().replace('\u202f', '')
            if 'от' in salary_info:
                vac_info['min_salary'] = int(salary_info.split(' ')[1])
            elif 'до' in salary_info:
                vac_info['max_salary'] = int(salary_info.split(' ')[1])
            elif '–' in salary_info:
                vac_info['min_salary'] = int(salary_info.split(' ')[0])
                vac_info['max_salary'] = int(salary_info.split(' ')[2])
            vac_info['salary_currency'] = salary_info.split(' ')[-1]
        except:
             pass

        company_info = vacancy.find('a', {'data-qa': 'vacancy-serp__vacancy-employer'})
        vac_info['company'] = company_info.getText().replace('\xa0', ' ')
        vac_info['comp_link'] = url + company_info.get('href')

        vac_info['city'] = vacancy.find('div', {'data-qa': 'vacancy-serp__vacancy-address'}).getText().replace('\xa02\xa0', '').replace('\xa01\xa0', '')

        all_vacancies.append(vac_info)

    print('Страница обработана')
    try:
        url2 = url + soup.find('a', {'data-qa': 'pager-next'}).get('href')
    except:
        break

Страница обработана
Страница обработана
Страница обработана


In [10]:
print(len(all_vacancies))
pprint(all_vacancies)

147
[{'city': 'Москва',
  'comp_link': 'https://hh.ru/employer/1887728?hhtmFrom=vacancy_search_list',
  'company': 'REVO Capital LLC',
  'link': 'https://kazan.hh.ru/vacancy/93344486?hhtmFrom=vacancy_search_list',
  'min_salary': 100000,
  'name': 'AI / ML специалист (стажёр)',
  'salary_currency': '₽'},
 {'city': 'Москва',
  'comp_link': 'https://hh.ru/employer/5686111?hhtmFrom=vacancy_search_list',
  'company': 'Olima',
  'link': 'https://kazan.hh.ru/vacancy/93540310?hhtmFrom=vacancy_search_list',
  'max_salary': 140000,
  'min_salary': 100000,
  'name': 'Младший аналитик (IT)',
  'salary_currency': '₽'},
 {'city': 'Москва, Маяковская и еще',
  'comp_link': 'https://hh.ru/employer/3127?dpt=mgfn-3127-it&hhtmFrom=vacancy_search_list',
  'company': 'ПАО МегаФон, IT',
  'link': 'https://kazan.hh.ru/vacancy/93379719?hhtmFrom=vacancy_search_list',
  'name': 'Data Analyst (Стажёр)'},
 {'city': 'Казань',
  'comp_link': 'https://hh.ru/employer/5686111?hhtmFrom=vacancy_search_list',
  'company

In [11]:
with open('vacancies.json', 'w', encoding='utf-8') as file:
    json.dump(all_vacancies, file)

In [12]:
df = pd.DataFrame(all_vacancies)

In [13]:
df

,name,link,min_salary,salary_currency,company,comp_link,city,max_salary
0,AI / ML специалист (стажёр),https://kazan.hh.ru/vacancy/93344486?hhtmFrom=...,100000.0,₽,REVO Capital LLC,https://hh.ru/employer/1887728?hhtmFrom=vacanc...,Москва,NaN
1,Младший аналитик (IT),https://kazan.hh.ru/vacancy/93540310?hhtmFrom=...,100000.0,₽,Olima,https://hh.ru/employer/5686111?hhtmFrom=vacanc...,Москва,140000.0
2,Data Analyst (Стажёр),https://kazan.hh.ru/vacancy/93379719?hhtmFrom=...,NaN,NaN,"ПАО МегаФон, IT",https://hh.ru/employer/3127?dpt=mgfn-3127-it&h...,"Москва, Маяковская и еще",NaN
3,Младший аналитик (IT),https://kazan.hh.ru/vacancy/92681061?hhtmFrom=...,100000.0,₽,Olima,https://hh.ru/employer/5686111?hhtmFrom=vacanc...,Казань,140000.0
4,Аналитик данных / стажер,https://kazan.hh.ru/vacancy/93537503?hhtmFrom=...,NaN,NaN,МТС Банк. Начало Карьеры,https://hh.ru/employer/4496?dpt=4496-4496-stud...,"Москва, Технопарк",NaN
...,...,...,...,...,...,...,...,...
142,Ассистент консультанта SAP (стажер),https://kazan.hh.ru/vacancy/93138448?hhtmFrom=...,NaN,NaN,"ООО ""Сибинтек-Софт""",https://hh.ru/employer/197135?dpt=197135-19713...,"Москва, Беговая",NaN
143,Младший аналитик отдела сопровождения сделок,https://kazan.hh.ru/vacancy/90803780?hhtmFrom=...,NaN,NaN,Б1 Аудиторские услуги,https://hh.ru/employer/788350?hhtmFrom=vacancy...,Минск,NaN
144,Стажер в команду отдела торгового маркетинга (...,https://kazan.hh.ru/vacancy/91832723?hhtmFrom=...,66000.0,₽,JTI,https://hh.ru/employer/162?hhtmFrom=vacancy_se...,Москва,66000.0
145,Product Manager (стажер),https://kazan.hh.ru/vacancy/93563871?hhtmFrom=...,NaN,NaN,ИП Graysar,https://hh.ru/employer/5279860?hhtmFrom=vacanc...,"Алматы, Абая и еще",NaN
